In [2]:
import os

c50_data = "/data/C50/C50train/"
authors = os.listdir(c50_data)
print(authors)

['AaronPressman', 'AlanCrosby', 'AlexanderSmith', 'BenjaminKangLim', 'BernardHickey', 'BradDorfman', 'DarrenSchuettler', 'DavidLawder', 'EdnaFernandes', 'EricAuchard', 'FumikoFujisaki', 'GrahamEarnshaw', 'HeatherScoffield', 'JanLopatka', 'JaneMacartney', 'JimGilchrist', 'JoWinterbottom', 'JoeOrtiz', 'JohnMastrini', 'JonathanBirt', 'KarlPenhaul', 'KeithWeir', 'KevinDrawbaugh', 'KevinMorrison', 'KirstinRidley', 'KouroshKarimkhany', 'LydiaZajc', "LynneO'Donnell", 'LynnleyBrowning', 'MarcelMichelson', 'MarkBendeich', 'MartinWolk', 'MatthewBunce', 'MichaelConnor', 'MureDickie', 'NickLouth', 'PatriciaCommins', 'PeterHumphrey', 'PierreTran', 'RobinSidel', 'RogerFillion', 'SamuelPerry', 'SarahDavison', 'ScottHillis', 'SimonCowell', 'TanEeLyn', 'TheresePoletti', 'TimFarrand', 'ToddNissen', 'WilliamKazer']


In [3]:
def get_texts_for_author(data_path, author_name):
    fpath = os.path.join(data_path, author_name)
    fnames = os.listdir(fpath)
    text_paths = [os.path.join(fpath, name) for name in fnames]
    texts = []
    for tp in text_paths:
        with open(tp) as f:
            s = f.read()
            texts.append(s)
    return texts

In [4]:
all_texts = [] 
all_labels = []
for author in authors:
    author_texts = get_texts_for_author(c50_data, author)
    all_texts += author_texts
    all_labels += [author] * len(author_texts)
    if len(author_texts) != 50:
        print(author, "not 50")

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from nltk.tokenize import word_tokenize
char = TfidfVectorizer(analyzer='char', ngram_range=(2,4), lowercase=True, min_df=5)
word = TfidfVectorizer(analyzer='word', ngram_range=(1,2), lowercase=False, min_df=5)
fu = FeatureUnion([
    ('char', char),
    ('word', word)
])

print("vectorizing")
Xs = fu.fit_transform(all_texts)
print("vectorized..")
ys = all_labels
cls = LinearSVC()
cls.fit(Xs, ys)

vectorizing
vectorized..


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [6]:
c50_test = "/data/C50/C50test/"
test_texts = [] 
test_labels = []
for author in authors:
    author_texts = get_texts_for_author(c50_test, author)
    test_texts += author_texts
    test_labels += [author] * len(author_texts)
    if len(author_texts) != 50:
        print(author, "not 50")

In [12]:
Xs = fu.transform(test_texts)

In [23]:
print(len(' '.join(all_texts).split(" "))/50)

print(len(' '.join(test_texts).split(" "))/50)

24379.56
24736.86


In [13]:
preds = cls.predict(Xs)
print(classification_report(test_labels, preds))

                   precision    recall  f1-score   support

    AaronPressman       0.87      0.96      0.91        50
       AlanCrosby       0.90      0.54      0.68        50
   AlexanderSmith       0.76      0.44      0.56        50
  BenjaminKangLim       0.44      0.32      0.37        50
    BernardHickey       0.97      0.72      0.83        50
      BradDorfman       0.69      0.94      0.80        50
 DarrenSchuettler       0.35      0.28      0.31        50
      DavidLawder       0.70      0.46      0.55        50
    EdnaFernandes       0.94      0.60      0.73        50
      EricAuchard       0.53      0.60      0.56        50
   FumikoFujisaki       0.94      1.00      0.97        50
   GrahamEarnshaw       0.73      0.94      0.82        50
 HeatherScoffield       0.33      0.40      0.36        50
       JanLopatka       0.60      0.42      0.49        50
    JaneMacartney       0.37      0.40      0.38        50
     JimGilchrist       0.94      1.00      0.97       

In [14]:
from sklearn.metrics import accuracy_score

print(accuracy_score(test_labels, preds))

0.7192


In [ ]:
# Cross validated results on train
print(classification_report(all_labels, npreds))

In [6]:
print(classification_report(all_labels, spreds))

NameError: name 'spreds' is not defined

In [42]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize
wvec = TfidfVectorizer(min_df=5, sublinear_tf=True, ngram_range=(1,2), lowercase=False, tokenizer=word_tokenize)
print("vectorizing")
Xs = wvec.fit_transform(all_texts)
print("vectorized..")
ys = all_labels
npreds = cross_val_predict(LinearSVC(), Xs, ys)

vectorizing
vectorized..


In [43]:
print(classification_report(all_labels, npreds))

                   precision    recall  f1-score   support

    AaronPressman       0.84      0.96      0.90        50
       AlanCrosby       0.60      0.66      0.63        50
   AlexanderSmith       0.73      0.48      0.58        50
  BenjaminKangLim       0.51      0.72      0.60        50
    BernardHickey       0.87      0.68      0.76        50
      BradDorfman       0.80      0.70      0.74        50
 DarrenSchuettler       0.85      0.90      0.87        50
      DavidLawder       0.68      0.82      0.75        50
    EdnaFernandes       0.77      0.60      0.67        50
      EricAuchard       0.71      0.68      0.69        50
   FumikoFujisaki       0.94      1.00      0.97        50
   GrahamEarnshaw       0.85      0.92      0.88        50
 HeatherScoffield       0.98      0.96      0.97        50
       JanLopatka       0.66      0.62      0.64        50
    JaneMacartney       0.64      0.42      0.51        50
     JimGilchrist       0.96      1.00      0.98       

In [12]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model


BASE_DIR = ''
GLOVE_DIR = BASE_DIR + '/glove.6B/'
TEXT_DATA_DIR = BASE_DIR + '/20_newsgroup/'
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
f = open("/data/glove/glove.6B.100d.txt")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')

texts = all_texts
label_to_id = {label: i for i, label in enumerate(sorted(list(set(all_labels))))}
labels = [label_to_id[label] for label in all_labels]

print('Found %s texts.' % len(texts))

Indexing word vectors.
Found 400000 word vectors.
Processing text dataset
Found 2500 texts.
Found 31090 unique tokens.
Shape of data tensor: (2500, 1000)
Shape of label tensor: (2500, 50)
Preparing embedding matrix.
Training model.
Train on 2000 samples, validate on 500 samples
Epoch 1/10
2000/2000 [==============================] - 30s - loss: 3.8967 - acc: 0.0225 - val_loss: 3.7825 - val_acc: 0.0360

In [ ]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(50, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
validation_data=(x_val, y_val))